In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
__idea__ = 'Filter Sharing And Parametric Tversky Loss Function'
__author__ = 'Dawood AL CHANTI'
__affiliation__ = 'LS2N-ECN'

In [ ]:
from PIL import Image
import numpy as np
import glob
import cv2
import os
from os.path import join
import random
from medpy.io import load
from medpy import metric

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
plt.rcParams['image.cmap'] = 'gist_earth'

In [ ]:
#Ignore warning in Jupyter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from __future__ import print_function, division, absolute_import, unicode_literals

In [ ]:
# define the location of .py helper module that we import
import sys
sys.path.insert(1, '/tf/JournalWork/')
sys.path.insert(1, '/tf/')

# Data Processing

In [ ]:
def crop_pad(img):    
    H,W = img.shape
    if H>=512 and W>=512:
        PadEdgesSize1_H = int(abs(512-H)/2.)
        PadEdgesSize2_H = int(abs(512-H) - PadEdgesSize1_H)
        PadEdgesSize1_W = int(abs(512-W)/2.)
        PadEdgesSize2_W = int(abs(512-W) - PadEdgesSize1_W)
        new=img[PadEdgesSize1_H:H-PadEdgesSize2_H,PadEdgesSize1_W:W-PadEdgesSize2_W]
        
    elif H<512 and W>=512:
        new = np.vstack((img[:,:512],np.zeros_like(img[:,:512])))
        HH,WW = new.shape
        PadEdgesSize1_H = int(abs(512-HH)/2.)
        PadEdgesSize2_H = int(abs(512-HH) - PadEdgesSize1_H)
        PadEdgesSize1_W = int(abs(512-WW)/2.)
        PadEdgesSize2_W = int(abs(512-WW) - PadEdgesSize1_W)
        new=new[:512,PadEdgesSize1_W:WW-PadEdgesSize2_W]
    
    elif H>=512 and W<512:    
        new = np.hstack((img[:512,:],np.zeros_like(img[:512,:])))
        HH,WW = new.shape
        PadEdgesSize1_H = int(abs(512-HH)/2.)
        PadEdgesSize2_H = int(abs(512-HH) - PadEdgesSize1_H)
        PadEdgesSize1_W = int(abs(512-WW)/2.)
        PadEdgesSize2_W = int(abs(512-WW) - PadEdgesSize1_W)
        new=new[PadEdgesSize1_H:HH-PadEdgesSize2_H,:512]
    
    elif H<512 and W<512:   
        new = np.hstack((img[:,:],np.zeros_like(img[:,:])))
        new = np.vstack((new[:,:512],np.zeros_like(new[:,:512])))
        new = new[:512,:512]
    return new

In [ ]:
def process_mask(mask_file_path):
    '''
    Return 3 masks in order SOL, GL and GM.
    Input: main path for the dataset: either Train/Val/or Testm i.e. '/tf/volumes/train/CAT_TH/masksX1.mha'
    '''
    
    #Read the data of formate (528, 640, 1574)
    mask_data,_ = load(mask_file_path)
    # Adjust the formate to (640, 528, 1574)
    mask_data = mask_data.transpose([1,0,2])

    Mask= list(map(lambda mask_time_step:  crop_pad(mask_time_step),
                   mask_data.transpose(2,0,1)))#,(512,640),

    # get the output for each muscle of formate timesteps x H x W 
    # data clip to replace values of 100 150 and 200 to 1
    mask_sol= np.array(list(map(lambda mask_time_step: np.clip(np.where(mask_time_step == 100, mask_time_step, 0), 
                                                               0, 1,
                                                               np.where(mask_time_step == 100, mask_time_step, 0))
                                ,Mask)))
    mask_gl= np.array(list(map(lambda mask_time_step:np.clip(np.where(mask_time_step == 200, mask_time_step, 0), 
                                                             0, 1,np.where(mask_time_step == 200, mask_time_step, 
                                                                           0)),Mask)))
    mask_gm= np.array(list(map(lambda mask_time_step: np.clip(np.where(mask_time_step == 150, 
                                                                       mask_time_step, 0), 0, 1,
                                                              np.where(mask_time_step == 150, mask_time_step, 0)),
                               Mask)))
    
    mask_sol = np.expand_dims(mask_sol,-1)
    mask_gl = np.expand_dims(mask_gl,-1)
    mask_gm = np.expand_dims(mask_gm,-1)
    
    
    # return the whole muscles on channel axis of order SOL GL and GM
    return np.concatenate([mask_sol,mask_gl,mask_gm],-1)

In [ ]:
#GL
def process_mask_GL(mask_file_path):
    '''
    Return 3 masks in order SOL, GL and GM.
    Input: main path for the dataset: either Train/Val/or Testm i.e. '/tf/volumes/train/CAT_TH/masksX1.mha'
    '''
    
    #Read the data of formate (528, 640, 1574)
    mask_data,_ = load(mask_file_path)
    # Adjust the formate to (640, 528, 1574)
    mask_data = mask_data.transpose([1,0,2])

    Mask= list(map(lambda mask_time_step:  crop_pad(mask_time_step),
                   mask_data.transpose(2,0,1)))#,(512,640),

        
    # get the output for each muscle of formate timesteps x H x W 
    # data clip to replace values of 100 150 and 200 to 1
    mask_sol= np.array(list(map(lambda mask_time_step: np.clip(np.where(mask_time_step == 200, mask_time_step, 0), 
                                                               0, 1,
                                                               np.where(mask_time_step == 200, mask_time_step, 0))
                                ,Mask)))

    mask_sol = np.expand_dims(mask_sol,-1)
 
    # Get the back ground of the annotated mask using the foreground annotation
    mask_sol=np.concatenate([mask_sol,1-mask_sol],-1)

    return mask_sol

In [ ]:
def process_data(data_file_path):
    '''
    Return 3 masks in order SOL, GL and GM.
    Input: main path for the dataset: either Train/Val/or Testm i.e. '/tf/volumes/train/CAT_TH/masksX1.mha'
    '''
    
    #Read the data of formate (528, 640, 1574)
    image_data,_ = load(data_file_path)
    # Adjust the formate to (640, 528, 1574)
    image_data = image_data.transpose([1,0,2])

    image_data= list(map(lambda mask_time_step:  crop_pad(mask_time_step),
                   image_data.transpose(2,0,1))) #,(512,640),
    
    #image_data= list(map(lambda image_data_step: image_data_step[:640,:512],image_data.transpose(2,0,1)))    
    
    return np.array(image_data)

In [ ]:
def Pull_data_from_path(path):
    data = process_data(path)
    # return normalized data
    # values from whole data
    mean_val = 19.027262640214904
    std_val = 34.175155632916
    
    data = (data-mean_val) / std_val
    # reshape to t,h,w,1
    return  np.expand_dims(data,-1)

In [ ]:
def Pull_data_from_path_Complete(path):
    data = process_data(path)
    # return normalized data
    # values from whole data
    mean_val = 19.027262640214904
    std_val = 34.175155632916
    
    data = (data-mean_val) / std_val
    # reshape to t,h,w,1
    return  np.expand_dims(data,-1)

In [ ]:
#GL
def process_mask_GL_Complete(mask_file_path):
    '''
    Return 3 masks in order SOL, GL and GM.
    Input: main path for the dataset: either Train/Val/or Testm i.e. '/tf/volumes/train/CAT_TH/masksX1.mha'
    '''
    
    #Read the data of formate (528, 640, 1574)
    mask_data,_ = load(mask_file_path)
    # Adjust the formate to (640, 528, 1574)
    mask_data = mask_data.transpose([1,0,2])

    Mask= list(map(lambda mask_time_step:  crop_pad(mask_time_step),
                   mask_data.transpose(2,0,1))) #(512,640)
    
    # get the output for each muscle of formate timesteps x H x W 
    # data clip to replace values of 100 150 and 200 to 1
    mask_sol= np.array(list(map(lambda mask_time_step: np.clip(np.where(mask_time_step == 200, mask_time_step, 0), 
                                                               0, 1,
                                                               np.where(mask_time_step == 200, mask_time_step, 0))
                                ,Mask)))

    mask_sol = np.expand_dims(mask_sol,-1)
 
    # Get the back ground of the annotated mask using the foreground annotation
    mask_sol=np.concatenate([mask_sol,1-mask_sol],-1)

    return mask_sol

In [ ]:
def Pull_mask_from_path(path):
    return process_mask(path)

In [ ]:
def Patient_name(diretoryPathforOnePatient):
    return diretoryPathforOnePatient.split('/')[4]

In [ ]:
# Define the path 
x_y_path = '/tf/volumes/train/'
# define the full path for each patient
Patient_folder_path = sorted(os.listdir(x_y_path))
Patient_folder_full_path = list(map(lambda v : str(join(x_y_path,v)) + '/', Patient_folder_path))

# Get the full path for the volume and the mask
DataTrainPath = list(map(lambda s : s+'x1.mha' , Patient_folder_full_path))
MasksTrainPath = list(map(lambda s : s+'masksX1.mha' , Patient_folder_full_path))

In [ ]:
print(DataTrainPath[0])
print(MasksTrainPath[0])

In [ ]:
# Return the patient Name
Patient_name(DataTrainPath[0])

In [ ]:
locals()[Patient_name(DataTrainPath[0]) + '_data']=Pull_data_from_path(DataTrainPath[0])
print(locals()[Patient_name(DataTrainPath[0]) + '_data'].shape)
locals()[Patient_name(DataTrainPath[0]) + '_mask']= process_mask(MasksTrainPath[0])
print(locals()[Patient_name(DataTrainPath[0]) + '_mask'].shape)

In [ ]:
# Plot each muscle alone
fig, ax = plt.subplots(1,4, sharey=True, figsize=(12,4))

ax[0].imshow(locals()[Patient_name(DataTrainPath[0]) + '_data'][560][:,:,0] , aspect="auto",cmap='gray')
ax[1].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask'][560][:,:,0] , aspect="auto",cmap='gray')
ax[2].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask'][560][:,:,1], aspect="auto",cmap='gray') 
ax[3].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask'][560][:,:,2] , aspect="auto",cmap='gray') 

ax[0].set_title('Volume')
ax[1].set_title('SOL')
ax[2].set_title('GL')
ax[3].set_title('GM')

In [ ]:
# only for SOL

In [ ]:
locals()[Patient_name(DataTrainPath[0]) + '_mask_GL']= process_mask_GL(MasksTrainPath[0])
print(locals()[Patient_name(DataTrainPath[0]) + '_mask_GL'].shape)

In [ ]:
# Plot each muscle alone
fig, ax = plt.subplots(1,2, sharey=True, figsize=(12,4))

ax[0].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask_GL'][700][:,:,0] , aspect="auto",cmap='gray')
ax[1].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask_GL'][700][:,:,1] , aspect="auto",cmap='gray')

ax[0].set_title('F')
ax[1].set_title('B')


# Data flip from the back to front (as a way of Data Augmentation and mimicing Bi directional)

In [ ]:
locals()[Patient_name(DataTrainPath[0]) + '_data_flip']=np.flip(Pull_data_from_path(DataTrainPath[0]),0)

locals()[Patient_name(DataTrainPath[0]) + '_mask_flip']= np.flip(process_mask(MasksTrainPath[0]),0)


In [ ]:
# Plot each muscle alone
fig, ax = plt.subplots(1,4, sharey=True, figsize=(12,4))

ax[0].imshow(locals()[Patient_name(DataTrainPath[0]) + '_data_flip'][700][:,:,0] , aspect="auto",cmap='gray')
ax[1].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask_flip'][700][:,:,0] , aspect="auto",cmap='gray')
ax[2].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask_flip'][700][:,:,1], aspect="auto",cmap='gray') 
ax[3].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask_flip'][700][:,:,2] , aspect="auto",cmap='gray') 

ax[0].set_title('Volume')
ax[1].set_title('SOL')
ax[2].set_title('GL')
ax[3].set_title('GM')

In [ ]:
# Plot each muscle alone
fig, ax = plt.subplots(1,4, sharey=True, figsize=(12,4))

ax[0].imshow(locals()[Patient_name(DataTrainPath[0]) + '_data'][700][:,:,0] -locals()[Patient_name(DataTrainPath[0]) + '_data_flip'][700][:,:,0], aspect="auto",cmap='gray')
ax[1].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask'][700][:,:,0]-locals()[Patient_name(DataTrainPath[0]) + '_mask_flip'][700][:,:,0] , aspect="auto",cmap='gray')
ax[2].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask'][700][:,:,1]-locals()[Patient_name(DataTrainPath[0]) + '_mask_flip'][700][:,:,1], aspect="auto",cmap='gray') 
ax[3].imshow(locals()[Patient_name(DataTrainPath[0]) + '_mask'][700][:,:,2]-locals()[Patient_name(DataTrainPath[0]) + '_mask_flip'][700][:,:,2] , aspect="auto",cmap='gray') 

ax[0].set_title('Volume')
ax[1].set_title('SOL')
ax[2].set_title('GL')
ax[3].set_title('GM')

In [ ]:
# Free Memory
del(locals()[Patient_name(DataTrainPath[0]) + '_data_flip'])
del(locals()[Patient_name(DataTrainPath[0]) + '_data'])
del(locals()[Patient_name(DataTrainPath[0]) + '_mask_flip'])
del(locals()[Patient_name(DataTrainPath[0]) + '_mask'])

# Graph Implementation of the Model

In [ ]:
import os,time,cv2
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib import rnn

In [ ]:
def Conv3DBlock(inputs, n_filters, kernel_size=[3, 3, 3], stride = [1,1,1],activation_fn=None):
        """
        Builds the 3d conv block 
        Apply successivly a 3D convolution, BatchNormalization and relu
        """
        # Skip pointwise by setting num_outputs=Non
        net = slim.conv3d(inputs, n_filters, kernel_size=kernel_size,stride=stride, activation_fn=activation_fn)
        net =  slim.layer_norm(net) #slim.batch_norm(net, fused=True)
        net = tf.nn.relu(net)
        return net

In [ ]:
def Conv3DBlockTranspose(inputs, n_filters, kernel_size=[3, 3, 3], stride = [1,1,1],activation_fn=None):
        """
        Builds the 3d conv transpose block 
        Apply successivly a 3D transpose convolution, BatchNormalization and relu
        """
        # Skip pointwise by setting num_outputs=Non
        net = slim.conv3d_transpose(inputs, n_filters, kernel_size=kernel_size,
                                    stride=stride, activation_fn=activation_fn)
        net =  slim.layer_norm(net) #slim.batch_norm(net, fused=True)
        net = tf.nn.relu(net)
        return net

In [ ]:
def AtrousSpatialPyramidPoolingModule_3D(inputs, depth=256):
    
    '''
    5D Tensor: batch, time, H, W, C
    '''
    feature_map_size = tf.shape(inputs)

    # Global average pooling
    image_features = tf.reduce_mean(inputs, [2, 3], keep_dims=True)

    image_features = slim.conv3d(image_features, depth, [1,1,1], activation_fn=None)

    image_features = tf.keras.layers.UpSampling3D(size=(1, int(inputs.shape[2]), 
                                                        int(inputs.shape[3])))(image_features)
    
    atrous_pool_block_1 = slim.conv3d(inputs, depth, [1, 1,1], activation_fn=None) # 3x3 filter reciptive field

    atrous_pool_block_6 = slim.conv3d(inputs, depth, [3, 3,3], rate=6, activation_fn=None)# 9x9

    atrous_pool_block_12 = slim.conv3d(inputs, depth, [3, 3,3], rate=12, activation_fn=None)# 15x15

    atrous_pool_block_18 = slim.conv3d(inputs, depth, [3, 3,3], rate=18, activation_fn=None)# 21x21

    net = tf.concat((image_features, atrous_pool_block_1, atrous_pool_block_6, 
                     atrous_pool_block_12, atrous_pool_block_18), axis=-1)

    return net

In [ ]:
def AtrousSpatialPyramidPoolingModule_3D_rate_9(inputs, depth=256):
    
    '''
    5D Tensor: batch, time, H, W, C
    '''
    feature_map_size = tf.shape(inputs)

    # Global average pooling
    image_features = tf.reduce_mean(inputs, [2, 3], keep_dims=True)

    image_features = slim.conv3d(image_features, depth, [1,1,1], activation_fn=None)
  

#     image_features = tf.transpose(tf.stack(list(map(lambda a: 
#                                                     tf.image.resize_bilinear(a, (feature_map_size[2], 
#                                                                                  feature_map_size[3])), 
#                                                     tf.unstack(tf.transpose(image_features,[1,0,2,3,4])))),0),
#                                   [1,0,2,3,4])
    
    
    image_features = tf.keras.layers.UpSampling3D(size=(1, int(inputs.shape[2]), 
                                                        int(inputs.shape[3])))(image_features)
    
    
    atrous_pool_block_1 = slim.conv3d(inputs, depth, [1, 1,1], activation_fn=None)# 3x3

    atrous_pool_block_6 = slim.conv3d(inputs, depth, [3, 3,3], rate=2, activation_fn=None)#5x5

    atrous_pool_block_12 = slim.conv3d(inputs, depth, [3, 3,3], rate=4, activation_fn=None)# 7x7

    atrous_pool_block_18 = slim.conv3d(inputs, depth, [3, 3,3], rate=6, activation_fn=None)# 9x9

    net = tf.concat((image_features, atrous_pool_block_1, atrous_pool_block_6, 
                     atrous_pool_block_12, atrous_pool_block_18), axis=-1)

    return net

In [ ]:
def AtrousSpatialPyramidPoolingModule_3D_rate_11(inputs, depth=256):
    
    '''
    5D Tensor: batch, time, H, W, C
    '''
    feature_map_size = tf.shape(inputs)

    # Global average pooling
    image_features = tf.reduce_mean(inputs, [2, 3], keep_dims=True)

    image_features = slim.conv3d(image_features, depth, [1,1,1], activation_fn=None)
  

#     image_features = tf.transpose(tf.stack(list(map(lambda a: 
#                                                     tf.image.resize_bilinear(a, (feature_map_size[2], 
#                                                                                  feature_map_size[3])), 
#                                                     tf.unstack(tf.transpose(image_features,[1,0,2,3,4])))),0),
#                                   [1,0,2,3,4])
    
    image_features = tf.keras.layers.UpSampling3D(size=(1, int(inputs.shape[2]), 
                                                        int(inputs.shape[3])))(image_features)
    
    
    atrous_pool_block_1 = slim.conv3d(inputs, depth, [1, 1,1], activation_fn=None)

    atrous_pool_block_6 = slim.conv3d(inputs, depth, [3, 3,3], rate=4, activation_fn=None) #7x7

    atrous_pool_block_12 = slim.conv3d(inputs, depth, [3, 3,3], rate=6, activation_fn=None)# 9x9

    atrous_pool_block_18 = slim.conv3d(inputs, depth, [3, 3,3], rate=8, activation_fn=None)# 11x11

    net = tf.concat((image_features, atrous_pool_block_1, atrous_pool_block_6, 
                     atrous_pool_block_12, atrous_pool_block_18), axis=-1)

    return net

In [ ]:
def AttentionRefinementModule_3D(inputs, n_filters):
    'for 3d data'
    # 3D Global average pooling
    net = tf.reduce_mean(inputs, [2, 3], keep_dims=True)
    net = slim.conv3d(net, n_filters, kernel_size=[1,1,1])
    net = slim.layer_norm(net)
    #net = slim.batch_norm(net, fused=True)
    net = tf.sigmoid(net)
    net = tf.multiply(inputs, net)
    return net

In [ ]:
def FeatureFusionModule(input_1, input_2, n_filters):
    
    inputs = tf.concat([input_1, input_2], axis=-1)
    inputs =Conv3DBlock(inputs, n_filters, kernel_size=[3, 3, 3], stride = [1,1,1],activation_fn=None)
    

    # Global average pooling
    net = tf.reduce_mean(inputs, [2, 3], keep_dims=True)
    
    net = slim.conv3d(net, n_filters, kernel_size=[1,1,1])
    net = tf.nn.relu(net)
    
    net = slim.conv3d(net, n_filters, kernel_size=[1,1,1])
    net = tf.sigmoid(net)

    net = tf.multiply(inputs, net)

    net = tf.add(inputs, net)
    return net


In [ ]:
def FeatureFusionModule_with_Stirde(input_1, input_2, n_filters):
    
    inputs = tf.concat([input_1, input_2], axis=-1)
    inputs =Conv3DBlock(inputs, n_filters, kernel_size=[3, 3, 3], stride = [1,2,2],activation_fn=None)
    

    # Global average pooling
    net = tf.reduce_mean(inputs, [2, 3], keep_dims=True)
    
    net = slim.conv3d(net, n_filters, kernel_size=[1,1,1])
    net = tf.nn.relu(net)
    
    net = slim.conv3d(net, n_filters, kernel_size=[1,1,1])
    net = tf.sigmoid(net)

    net = tf.multiply(inputs, net)

    net = tf.add(inputs, net)
    return net


In [ ]:
def iou_coe(output, target, threshold=0.5, axis=(1, 2, 3,4), smooth=1e-5):
    """Non-differentiable Intersection over Union (IoU) for comparing the similarity 
    """
    pre = tf.cast(output > threshold, dtype=tf.float32)
    truth = tf.cast(target > threshold, dtype=tf.float32)
    inse = tf.reduce_sum(tf.multiply(pre, truth), axis=axis)  # AND
    union = tf.reduce_sum(tf.cast(tf.add(pre, truth) >= 1, dtype=tf.float32), axis=axis)  # OR
    batch_iou = (inse + smooth) / (union + smooth)
    iou = tf.reduce_mean(batch_iou, name='iou_coe')
    return iou  # , pre, truth, inse, union



In [ ]:
def dice_hard_coe(output, target, threshold=0.5, axis=(1, 2, 3,4), smooth=1e-5):
    """Non-differentiable Sørensen–Dice coefficient for comparing the similarity
    """
    output = tf.cast(output > threshold, dtype=tf.float32)
    target = tf.cast(target > threshold, dtype=tf.float32)
    inse = tf.reduce_sum(tf.multiply(output, target), axis=axis)
    l = tf.reduce_sum(output, axis=axis)
    r = tf.reduce_sum(target, axis=axis)
    hard_dice = (2. * inse + smooth) / (l + r + smooth)
    ##
    hard_dice = tf.reduce_mean(hard_dice, name='hard_dice')
    return hard_dice

In [ ]:
K = tf.keras.backend

def confusion(y_true, y_pred):
    smooth=1
    y_pred_pos = K.clip(y_pred, 0, 1)
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.clip(y_true, 0, 1)
    y_neg = 1 - y_pos
    tp = K.sum(y_pos * y_pred_pos)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg) 
    prec = (tp + smooth)/(tp+fp+smooth)
    recall = (tp+smooth)/(tp+fn+smooth)
    return prec, recall


### Rest the Graph and build it

In [ ]:
tf.reset_default_graph()

## Define the parameters

In [ ]:
time_step = None # whatever the depth of the volume, we use a sliding window of T
H=512
W=512
C = 1 # number of input channels
num_classes = 2  # or 6 # related to SOl GL and GM
n_class = 2 # of 6 if all treated at the same time

## Define the place holder that takes in the data

In [ ]:
# define the input stracture: 5D Tensor

x = tf.placeholder("float", shape=[None,time_step, H, W, C ], name="x")
y = tf.placeholder("float", shape=[None,time_step,H, W,num_classes], name="y")

# define the placeholder for dropout
keep_prob = tf.placeholder(tf.float32, name="dropout_probability")
keep_prob_input = tf.placeholder(tf.float32, name="dropout_probability_input")
keep_prob_skip = tf.placeholder(dtype=tf.float32,name='SkipDropout')

print('Input Structure: ',x)
print('Target Structure: ',y)

In [ ]:
keep_prob_alpha = tf.placeholder(tf.float32, name="drop_alpha")
keep_prob_beta = tf.placeholder(tf.float32, name="drop_beta")
keep_prob_tp = tf.placeholder(tf.float32, name="drop_tp")

In [ ]:
# defined as variables ?
Batch_size = tf.shape(x)[0]
time_size = tf.shape(x)[1]

In [ ]:
def Downsampling_2D(inputs,scale1,scale2):
    return tf.image.resize_bilinear(inputs, size=[scale1,scale2])

def Down_sample_3D(input_layer,scale1,scale2):
    '''
    TODO : 10 should be changed to new time scale 10 30 or 200 
    '''
    unpol_layer = list(map(lambda layer: Downsampling_2D(layer,scale1,scale2), tf.unstack(input_layer, 
                                                                                  int(input_layer.get_shape()[1]), 
                                                                                  1)))
    return tf.transpose(tf.stack(unpol_layer),[1,0,2,3,4])


## Skip and unpool 

In [ ]:
def skip(layer, end_point): return tf.concat([layer, end_point], axis=4)
def unpool(layer): return tf.image.resize_nearest_neighbor(layer, 
                                                           [2*int(layer.get_shape()[1]), 
                                                            2*int(layer.get_shape()[2])])
# unpool based on 2D data
def unpool_3D_2(input_layer):
    unpol_layer = list(map(lambda layer: unpool(layer), tf.unstack(input_layer, 
                                                                   int(input_layer.get_shape()[1]), 1)))
    return tf.transpose(tf.stack(unpol_layer),[1,0,2,3,4])


def unpool_3D(input_layer):
    
    unpol_layer = tf.keras.layers.UpSampling3D(size=(1, 2,2))(input_layer)
    return unpol_layer


## Build the Encoder Target Stream where no annotation

In [ ]:
# Re-define for 5D Tensor, here we are dealing with 5D: axis=3 become axis=4 last channel
def pixel_wise_softmax(output_map):
    return tf.nn.softmax(output_map)

# Encoding Path

In [ ]:
#rate=6
#strides=[1, 1, 2, 2, 1], 
Reuse_Layer_1 = False
with tf.variable_scope("layer_1",reuse=Reuse_Layer_1) as scope:
    
    Layer_1 = slim.conv3d(x, 4, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_1 = slim.layer_norm(Layer_1) #slim.batch_norm(net, fused=True)
    Layer_1 = tf.nn.relu(Layer_1)
    
    Layer_1 = tf.nn.max_pool3d(Layer_1, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_1p')
    Layer_1 =  tf.nn.dropout(Layer_1,keep_prob_skip)
    print(Layer_1)

In [ ]:
#rate=6
Reuse_Layer_1 = True
with tf.variable_scope("layer_1",reuse=Reuse_Layer_1) as scope:
    
    Layer_1dr2 = slim.conv3d(x, 4, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], rate=3,
                          activation_fn=None)
    Layer_1dr2 = slim.layer_norm(Layer_1dr2) #slim.batch_norm(net, fused=True)
    Layer_1dr2 = tf.nn.relu(Layer_1dr2)
    
    Layer_1dr2 = tf.nn.max_pool3d(Layer_1dr2, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_1p')
    
    Layer_1dr2 =  tf.nn.dropout(Layer_1dr2,keep_prob_skip)
    print(Layer_1dr2)

In [ ]:
#rate=6
Reuse_Layer_1 = True
with tf.variable_scope("layer_1",reuse=Reuse_Layer_1) as scope:
    
    Layer_1dr3 = slim.conv3d(x, 4, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], rate=7,
                          activation_fn=None)
    Layer_1dr3 = slim.layer_norm(Layer_1dr3) #slim.batch_norm(net, fused=True)
    Layer_1dr3 = tf.nn.relu(Layer_1dr3)
    
    Layer_1dr3 = tf.nn.max_pool3d(Layer_1dr3, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_1p')
    Layer_1dr3 =  tf.nn.dropout(Layer_1dr3,keep_prob_skip)
    print(Layer_1dr3)

In [ ]:
#rate=6
Reuse_Layer_1 = True
with tf.variable_scope("layer_1",reuse=Reuse_Layer_1) as scope:
    
    Layer_1dr4 = slim.conv3d(x, 4, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], rate=9,
                          activation_fn=None)
    Layer_1dr4 = slim.layer_norm(Layer_1dr4) #slim.batch_norm(net, fused=True)
    Layer_1dr4 = tf.nn.relu(Layer_1dr4)
    
    Layer_1dr4 = tf.nn.max_pool3d(Layer_1dr4, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_1p')
    Layer_1dr4 =  tf.nn.dropout(Layer_1dr4,keep_prob_skip)
    print(Layer_1dr4)

In [ ]:
Layer_1 = tf.concat([tf.concat([tf.concat([Layer_1,Layer_1dr2],-1),Layer_1dr3],-1),Layer_1dr4],-1)

In [ ]:
Layer_1

In [ ]:
Reuse_Layer_2 = False
with tf.variable_scope("layer_2",reuse=Reuse_Layer_2) as scope:

    Layer_2 = slim.conv3d(Layer_1, 8, kernel_size=[3, 3, 3],stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_2 = slim.layer_norm(Layer_2) #slim.batch_norm(net, fused=True)
    Layer_2 = tf.nn.relu(Layer_2)
    Layer_2 = tf.nn.max_pool3d(Layer_2, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_2p')
    Layer_2 =  tf.nn.dropout(Layer_2,keep_prob_skip)
    print(Layer_2)

In [ ]:
Reuse_Layer_2 = True
with tf.variable_scope("layer_2",reuse=Reuse_Layer_2) as scope:
    Layer_2r2 = slim.conv3d(Layer_1, 8, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=3,
                          activation_fn=None)
    Layer_2r2 = slim.layer_norm(Layer_2r2) #slim.batch_norm(net, fused=True)
    Layer_2r2 = tf.nn.relu(Layer_2r2)
    Layer_2r2 = tf.nn.max_pool3d(Layer_2r2, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_2p')
    Layer_2r2 =  tf.nn.dropout(Layer_2r2,keep_prob_skip)
    print(Layer_2r2)

In [ ]:
Reuse_Layer_2 = True
with tf.variable_scope("layer_2",reuse=Reuse_Layer_2) as scope:
    
    Layer_2r3 = slim.conv3d(Layer_1, 8, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=7,
                          activation_fn=None)
    Layer_2r3 = slim.layer_norm(Layer_2r3) #slim.batch_norm(net, fused=True)
    Layer_2r3 = tf.nn.relu(Layer_2r3)
    Layer_2r3 = tf.nn.max_pool3d(Layer_2r3, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_2p')
    Layer_2r3 =  tf.nn.dropout(Layer_2r3,keep_prob_skip)
    print(Layer_2r3)

In [ ]:
Reuse_Layer_2 = True
with tf.variable_scope("layer_2",reuse=Reuse_Layer_2) as scope:

    Layer_2r4 = slim.conv3d(Layer_1, 8, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=9,
                          activation_fn=None)
    Layer_2r4 = slim.layer_norm(Layer_2r4) #slim.batch_norm(net, fused=True)
    Layer_2r4 = tf.nn.relu(Layer_2r4)
    Layer_2r4 = tf.nn.max_pool3d(Layer_2r4, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_2p')
    Layer_2r4 =  tf.nn.dropout(Layer_2r4,keep_prob_skip)
    print(Layer_2r4)

In [ ]:
Layer_2 = tf.concat([tf.concat([tf.concat([Layer_2,Layer_2r2],-1),Layer_2r3],-1),Layer_2r4],-1)
Layer_2

In [ ]:
Reuse_Layer_3 = False
with tf.variable_scope("layer_3",reuse=Reuse_Layer_3) as scope:
    Layer_3_2 = slim.conv3d(Layer_2, 16, kernel_size=[3, 3, 3],stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_3_2 = slim.layer_norm(Layer_3_2) #slim.batch_norm(net, fused=True)
    Layer_3_2 = tf.nn.relu(Layer_3_2)
    Layer_3_2 = tf.nn.max_pool3d(Layer_3_2, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_3p')
    Layer_3_2 =  tf.nn.dropout(Layer_3_2,keep_prob_skip)
    print(Layer_3_2)

In [ ]:
Reuse_Layer_3 = True
with tf.variable_scope("layer_3",reuse=Reuse_Layer_3) as scope:
    

    Layer_3_2r2 = slim.conv3d(Layer_2, 16, kernel_size=[3, 3, 3],stride=[1, 1, 1],  rate=3,
                          activation_fn=None)
    Layer_3_2r2 = slim.layer_norm(Layer_3_2r2) #slim.batch_norm(net, fused=True)
    Layer_3_2r2 = tf.nn.relu(Layer_3_2r2)
    Layer_3_2r2 = tf.nn.max_pool3d(Layer_3_2r2, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_3p')
    Layer_3_2r2 =  tf.nn.dropout(Layer_3_2r2,keep_prob_skip)
    print(Layer_3_2r2)

In [ ]:
Reuse_Layer_3 = True
with tf.variable_scope("layer_3",reuse=Reuse_Layer_3) as scope:

    Layer_3_2r3 = slim.conv3d(Layer_2, 16, kernel_size=[3, 3, 3],stride=[1, 1, 1],  rate=7,
                          activation_fn=None)
    Layer_3_2r3 = slim.layer_norm(Layer_3_2r3) #slim.batch_norm(net, fused=True)
    Layer_3_2r3 = tf.nn.relu(Layer_3_2r3)
    Layer_3_2r3 = tf.nn.max_pool3d(Layer_3_2r3, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_3p')
    Layer_3_2r3 =  tf.nn.dropout(Layer_3_2r3,keep_prob_skip)
    print(Layer_3_2r3)

In [ ]:
Reuse_Layer_3 = True
with tf.variable_scope("layer_3",reuse=Reuse_Layer_3) as scope:
    Layer_3_2r4 = slim.conv3d(Layer_2, 16, kernel_size=[3, 3, 3],stride=[1, 1, 1],  rate=9,
                          activation_fn=None)
    Layer_3_2r4 = slim.layer_norm(Layer_3_2r4) #slim.batch_norm(net, fused=True)
    Layer_3_2r4 = tf.nn.relu(Layer_3_2r4)
    Layer_3_2r4 = tf.nn.max_pool3d(Layer_3_2r4, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_3p')
    Layer_3_2r4 =  tf.nn.dropout(Layer_3_2r4,keep_prob_skip)
    print(Layer_3_2r4)

In [ ]:
Layer_3_2 = tf.concat([tf.concat([tf.concat([Layer_3_2,Layer_3_2r2],-1),Layer_3_2r3],-1),Layer_3_2r4],-1)
Layer_3_2


In [ ]:
Reuse_Layer_4 = False
with tf.variable_scope("layer_4",reuse=Reuse_Layer_4) as scope:
    
    Layer_4 = slim.conv3d(Layer_3_2, 32, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_4 = slim.layer_norm(Layer_4) 
    Layer_4 = tf.nn.relu(Layer_4)
    
    Layer_4 = tf.nn.max_pool3d(Layer_4, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_4p')
    Layer_4 =  tf.nn.dropout(Layer_4,keep_prob_skip)
    print(Layer_4)

In [ ]:
Reuse_Layer_4 = True
with tf.variable_scope("layer_4",reuse=Reuse_Layer_4) as scope:
    
    Layer_4r2 = slim.conv3d(Layer_3_2, 32, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], rate=3,
                          activation_fn=None)
    Layer_4r2 = slim.layer_norm(Layer_4r2) 
    Layer_4r2 = tf.nn.relu(Layer_4r2)
    
    Layer_4r2 = tf.nn.max_pool3d(Layer_4r2, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_4p')
    Layer_4r2 =  tf.nn.dropout(Layer_4r2,keep_prob_skip)
    print(Layer_4r2)

In [ ]:
Reuse_Layer_4 = True
with tf.variable_scope("layer_4",reuse=Reuse_Layer_4) as scope:
    
    Layer_4r3 = slim.conv3d(Layer_3_2, 32, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], rate=7,
                          activation_fn=None)
    Layer_4r3 = slim.layer_norm(Layer_4r3) 
    Layer_4r3 = tf.nn.relu(Layer_4r3)
    
    Layer_4r3 = tf.nn.max_pool3d(Layer_4r3, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_4p')
    Layer_4r3 =  tf.nn.dropout(Layer_4r3,keep_prob_skip)
    print(Layer_4r3)

In [ ]:
Reuse_Layer_4 = True
with tf.variable_scope("layer_4",reuse=Reuse_Layer_4) as scope:
    
    Layer_4r4 = slim.conv3d(Layer_3_2, 32, kernel_size=[ 3, 3, 3],stride=[1, 1, 1], rate=9,
                          activation_fn=None)
    Layer_4r4 = slim.layer_norm(Layer_4r4) 
    Layer_4r4 = tf.nn.relu(Layer_4r4)
    
    Layer_4r4 = tf.nn.max_pool3d(Layer_4r4, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_4p')
    Layer_4r4 =  tf.nn.dropout(Layer_4r4,keep_prob_skip)
    print(Layer_4r4)

In [ ]:

Layer_4 = tf.concat([tf.concat([tf.concat([Layer_4,Layer_4r2],-1),Layer_4r3],-1),Layer_4r4],-1)
Layer_4


In [ ]:
Reuse_Layer_5 = False
with tf.variable_scope("layer_5",reuse=Reuse_Layer_5) as scope:
    
    Layer_5 = slim.conv3d(Layer_4, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_5 = slim.layer_norm(Layer_5) #slim.batch_norm(net, fused=True)
    Layer_5 = tf.nn.relu(Layer_5)
    Layer_5 = tf.nn.max_pool3d(Layer_5, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_5p')
    Layer_5 =  tf.nn.dropout(Layer_5,keep_prob_skip)
    print(Layer_5)

In [ ]:
Reuse_Layer_5 = True
with tf.variable_scope("layer_5",reuse=Reuse_Layer_5) as scope:
    
    Layer_5r2 = slim.conv3d(Layer_4, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=3,
                          activation_fn=None)
    Layer_5r2 = slim.layer_norm(Layer_5r2) #slim.batch_norm(net, fused=True)
    Layer_5r2 = tf.nn.relu(Layer_5r2)
    Layer_5r2 = tf.nn.max_pool3d(Layer_5r2, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_5p')
    Layer_5r2 =  tf.nn.dropout(Layer_5r2,keep_prob_skip)
    print(Layer_5r2)

In [ ]:
Reuse_Layer_5 = True
with tf.variable_scope("layer_5",reuse=Reuse_Layer_5) as scope:
    
    Layer_5r3 = slim.conv3d(Layer_4, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=7,
                          activation_fn=None)
    Layer_5r3 = slim.layer_norm(Layer_5r3) #slim.batch_norm(net, fused=True)
    Layer_5r3 = tf.nn.relu(Layer_5r3)
    Layer_5r3 = tf.nn.max_pool3d(Layer_5r3, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_5p')
    Layer_5r3 =  tf.nn.dropout(Layer_5r3,keep_prob_skip)
    print(Layer_5r3)

In [ ]:
Reuse_Layer_5 = True
with tf.variable_scope("layer_5",reuse=Reuse_Layer_5) as scope:
    
    Layer_5r4 = slim.conv3d(Layer_4, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=9,
                          activation_fn=None)
    Layer_5r4 = slim.layer_norm(Layer_5r4) #slim.batch_norm(net, fused=True)
    Layer_5r4 = tf.nn.relu(Layer_5r4)
    Layer_5r4 = tf.nn.max_pool3d(Layer_5r4, strides=[1, 1, 2, 2, 1], 
                                        ksize=[1, 3, 3, 3, 1], padding='SAME', name='Layer_5p')
    Layer_5r4 =  tf.nn.dropout(Layer_5r4,keep_prob_skip)
    print(Layer_5r4)

In [ ]:
Layer_5 = tf.concat([tf.concat([tf.concat([Layer_5,Layer_5r2],-1),Layer_5r3],-1),Layer_5r4],-1)
Layer_5


# Decoding

In [ ]:
Reuse_Layer_6 = False
with tf.variable_scope("layer_6",reuse=Reuse_Layer_6) as scope:
    
    Layer_6_5 = slim.conv3d(Layer_5, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_6_5 = slim.layer_norm(Layer_6_5) #slim.batch_norm(net, fused=True)
    Layer_6_5 = tf.nn.relu(Layer_6_5)
    Layer_6_5 = unpool_3D(Layer_6_5)
    Layer_6_5 =  tf.nn.dropout(Layer_6_5,keep_prob_skip)
    print(Layer_6_5)

In [ ]:
Reuse_Layer_6 = True
with tf.variable_scope("layer_6",reuse=Reuse_Layer_6) as scope:
    
    Layer_6_5r2 = slim.conv3d(Layer_5, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=3,
                          activation_fn=None)
    Layer_6_5r2 = slim.layer_norm(Layer_6_5r2) #slim.batch_norm(net, fused=True)
    Layer_6_5r2 = tf.nn.relu(Layer_6_5r2)
    Layer_6_5r2 = unpool_3D(Layer_6_5r2)
    Layer_6_5r2 =  tf.nn.dropout(Layer_6_5r2,keep_prob_skip)
    print(Layer_6_5r2)

In [ ]:
Reuse_Layer_6 = True
with tf.variable_scope("layer_6",reuse=Reuse_Layer_6) as scope:
    
    Layer_6_5r3 = slim.conv3d(Layer_5, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=7,
                          activation_fn=None)
    Layer_6_5r3 = slim.layer_norm(Layer_6_5r3) #slim.batch_norm(net, fused=True)
    Layer_6_5r3 = tf.nn.relu(Layer_6_5r3)
    Layer_6_5r3 = unpool_3D(Layer_6_5r3)
    Layer_6_5r3 =  tf.nn.dropout(Layer_6_5r3,keep_prob_skip)
    print(Layer_6_5r3)

In [ ]:
Reuse_Layer_6 = True
with tf.variable_scope("layer_6",reuse=Reuse_Layer_6) as scope:
    
    Layer_6_5r4 = slim.conv3d(Layer_5, 64, kernel_size=[3, 3, 3],stride=[1, 1, 1], rate=9,
                          activation_fn=None)
    Layer_6_5r4 = slim.layer_norm(Layer_6_5r4) #slim.batch_norm(net, fused=True)
    Layer_6_5r4 = tf.nn.relu(Layer_6_5r4)
    Layer_6_5r4 = unpool_3D(Layer_6_5r4)
    Layer_6_5r4 =  tf.nn.dropout(Layer_6_5r4,keep_prob_skip)
    print(Layer_6_5r4)

In [ ]:
Layer_6_5 = tf.concat([tf.concat([tf.concat([Layer_6_5,Layer_6_5r2],-1),Layer_6_5r3],-1),Layer_6_5r4],-1)
Layer_6_5

In [ ]:
Reuse_Layer_7 = False
with tf.variable_scope("layer_7",reuse=Reuse_Layer_7) as scope:
    Layer_7 = slim.conv3d(Layer_6_5, 32, 
                          kernel_size=[ 3, 3, 3],
                          stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_7 = slim.layer_norm(Layer_7) 
    Layer_7 = tf.nn.relu(Layer_7)
    Layer_7 = unpool_3D(Layer_7)
    Layer_7 =  tf.nn.dropout(Layer_7,keep_prob_skip)
    print(Layer_7)

In [ ]:
Reuse_Layer_7 = True
with tf.variable_scope("layer_7",reuse=Reuse_Layer_7) as scope:
    Layer_7r2 = slim.conv3d(Layer_6_5, 32, 
                          kernel_size=[ 3, 3, 3],rate=3,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_7r2 = slim.layer_norm(Layer_7r2) 
    Layer_7r2 = tf.nn.relu(Layer_7r2)
    Layer_7r2 = unpool_3D(Layer_7r2)
    Layer_7r2 =  tf.nn.dropout(Layer_7r2,keep_prob_skip)
    print(Layer_7r2)

In [ ]:
Reuse_Layer_7 = True
with tf.variable_scope("layer_7",reuse=Reuse_Layer_7) as scope:
    Layer_7r3 = slim.conv3d(Layer_6_5, 32, 
                          kernel_size=[ 3, 3, 3],rate=7,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_7r3 = slim.layer_norm(Layer_7r3) 
    Layer_7r3 = tf.nn.relu(Layer_7r3)
    Layer_7r3 = unpool_3D(Layer_7r3)
    Layer_7r3 =  tf.nn.dropout(Layer_7r3,keep_prob_skip)
    print(Layer_7r3)

In [ ]:
Reuse_Layer_7 = True
with tf.variable_scope("layer_7",reuse=Reuse_Layer_7) as scope:
    Layer_7r4 = slim.conv3d(Layer_6_5, 32, 
                          kernel_size=[ 3, 3, 3], rate=9,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_7r4 = slim.layer_norm(Layer_7r4) 
    Layer_7r4 = tf.nn.relu(Layer_7r4)
    Layer_7r4 = unpool_3D(Layer_7r4)
    Layer_7r4 =  tf.nn.dropout(Layer_7r4,keep_prob_skip)
    print(Layer_7r4)

In [ ]:
Layer_7 = tf.concat([tf.concat([tf.concat([Layer_7,Layer_7r2],-1),Layer_7r3],-1),Layer_7r4],-1)
Layer_7

In [ ]:
Reuse_Layer_8 = False
with tf.variable_scope("layer_8",reuse=Reuse_Layer_8) as scope:
    
    Layer_8 = slim.conv3d(Layer_7, 16, 
                          kernel_size=[ 3, 3, 3],
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_8 = slim.layer_norm(Layer_8) 
    Layer_8 = tf.nn.relu(Layer_8)
    Layer_8 = unpool_3D(Layer_8)
    Layer_8 =  tf.nn.dropout(Layer_8,keep_prob_skip)
    print(Layer_8)

In [ ]:
Reuse_Layer_8 = True
with tf.variable_scope("layer_8",reuse=Reuse_Layer_8) as scope:
    
    Layer_8r2 = slim.conv3d(Layer_7, 16, 
                          kernel_size=[ 3, 3, 3],rate=3,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_8r2 = slim.layer_norm(Layer_8r2) 
    Layer_8r2 = tf.nn.relu(Layer_8r2)
    Layer_8r2 = unpool_3D(Layer_8r2)
    Layer_8r2 =  tf.nn.dropout(Layer_8r2,keep_prob_skip)
    print(Layer_8r2)

In [ ]:
Reuse_Layer_8 = True
with tf.variable_scope("layer_8",reuse=Reuse_Layer_8) as scope:
    
    Layer_8r3 = slim.conv3d(Layer_7, 16, 
                          kernel_size=[ 3, 3, 3],rate=7,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_8r3 = slim.layer_norm(Layer_8r3) 
    Layer_8r3 = tf.nn.relu(Layer_8r3)
    Layer_8r3 = unpool_3D(Layer_8r3)
    Layer_8r3 =  tf.nn.dropout(Layer_8r3,keep_prob_skip)
    print(Layer_8r3)

In [ ]:
Reuse_Layer_8 = True
with tf.variable_scope("layer_8",reuse=Reuse_Layer_8) as scope:
    
    Layer_8r4 = slim.conv3d(Layer_7, 16, 
                          kernel_size=[ 3, 3, 3],rate=9,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_8r4 = slim.layer_norm(Layer_8r4) 
    Layer_8r4 = tf.nn.relu(Layer_8r4)
    Layer_8r4 = unpool_3D(Layer_8r4)
    Layer_8r4 =  tf.nn.dropout(Layer_8r4,keep_prob_skip)
    print(Layer_8r4)

In [ ]:
Layer_8 = tf.concat([tf.concat([tf.concat([Layer_8,Layer_8r2],-1),Layer_8r3],-1),Layer_8r4],-1)
Layer_8

In [ ]:
Reuse_Layer_9 = False
with tf.variable_scope("layer_9",reuse=Reuse_Layer_9) as scope:
    
    Layer_9_8 = slim.conv3d(Layer_8, 8, 
                          kernel_size=[ 3, 3, 3],
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_9_8 = slim.layer_norm(Layer_9_8) 
    Layer_9_8 = tf.nn.relu(Layer_9_8)
    Layer_9_8 = unpool_3D(Layer_9_8)
    Layer_9_8 =  tf.nn.dropout(Layer_9_8,keep_prob_skip)
    print(Layer_9_8)

In [ ]:
Reuse_Layer_9 = True
with tf.variable_scope("layer_9",reuse=Reuse_Layer_9) as scope:
    
    Layer_9_8r2 = slim.conv3d(Layer_8, 8, 
                          kernel_size=[ 3, 3, 3],rate=3,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_9_8r2 = slim.layer_norm(Layer_9_8r2) 
    Layer_9_8r2 = tf.nn.relu(Layer_9_8r2)
    Layer_9_8r2 = unpool_3D(Layer_9_8r2)
    Layer_9_8r2 =  tf.nn.dropout(Layer_9_8r2,keep_prob_skip)
    print(Layer_9_8r2)

In [ ]:
Reuse_Layer_9 = True
with tf.variable_scope("layer_9",reuse=Reuse_Layer_9) as scope:
    
    Layer_9_8r3 = slim.conv3d(Layer_8, 8, 
                          kernel_size=[ 3, 3, 3],rate=7,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_9_8r3 = slim.layer_norm(Layer_9_8r3) 
    Layer_9_8r3 = tf.nn.relu(Layer_9_8r3)
    Layer_9_8r3 = unpool_3D(Layer_9_8r3)
    Layer_9_8r3 =  tf.nn.dropout(Layer_9_8r3,keep_prob_skip)
    print(Layer_9_8r3)

In [ ]:
Reuse_Layer_9 = True
with tf.variable_scope("layer_9",reuse=Reuse_Layer_9) as scope:
    
    Layer_9_8r4 = slim.conv3d(Layer_8, 8, 
                          kernel_size=[ 3, 3, 3],rate=9,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    Layer_9_8r4 = slim.layer_norm(Layer_9_8r4) 
    Layer_9_8r4 = tf.nn.relu(Layer_9_8r4)
    Layer_9_8r4 = unpool_3D(Layer_9_8r4)
    Layer_9_8r4 =  tf.nn.dropout(Layer_9_8r4,keep_prob_skip)
    print(Layer_9_8r4)

In [ ]:
Layer_9_8 = tf.concat([tf.concat([tf.concat([Layer_9_8,Layer_9_8r2],-1),Layer_9_8r3],-1),Layer_9_8r4],-1)
Layer_9_8

In [ ]:
Layer_9_8

In [ ]:
Reuse_Layer_10 = False
with tf.variable_scope("layer_10",reuse=Reuse_Layer_10) as scope:
    
    Layer_10 = slim.conv3d(Layer_9_8, 4, 
                          kernel_size=[ 3, 3, 3],
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_10 = slim.layer_norm(Layer_10) 
    Layer_10 = tf.nn.relu(Layer_10)

    Layer_10 = unpool_3D(Layer_10)
    Layer_10 =  tf.nn.dropout(Layer_10,keep_prob_skip)
    
    print(Layer_10)

In [ ]:
Reuse_Layer_10 = True
with tf.variable_scope("layer_10",reuse=Reuse_Layer_10) as scope:
    
    Layer_10r2 = slim.conv3d(Layer_9_8, 4, 
                          kernel_size=[ 3, 3, 3],rate=3,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_10r2 = slim.layer_norm(Layer_10r2) 
    Layer_10r2 = tf.nn.relu(Layer_10r2)

    Layer_10r2 = unpool_3D(Layer_10r2)
    Layer_10r2 =  tf.nn.dropout(Layer_10r2,keep_prob_skip)
    
    print(Layer_10r2)

In [ ]:
Reuse_Layer_10 = True
with tf.variable_scope("layer_10",reuse=Reuse_Layer_10) as scope:
    
    Layer_10r3 = slim.conv3d(Layer_9_8, 4, 
                          kernel_size=[ 3, 3, 3],rate=7,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_10r3 = slim.layer_norm(Layer_10r3) 
    Layer_10r3 = tf.nn.relu(Layer_10r3)

    Layer_10r3 = unpool_3D(Layer_10r3)
    Layer_10r3 =  tf.nn.dropout(Layer_10r3,keep_prob_skip)
    
    print(Layer_10r3)

In [ ]:
Reuse_Layer_10 = True
with tf.variable_scope("layer_10",reuse=Reuse_Layer_10) as scope:
    
    Layer_10r4 = slim.conv3d(Layer_9_8, 4, 
                          kernel_size=[ 3, 3, 3],rate=9,
                          stride=[1, 1, 1], 
                          activation_fn=None)
    
    Layer_10r4 = slim.layer_norm(Layer_10r4) 
    Layer_10r4 = tf.nn.relu(Layer_10r4)

    Layer_10r4 = unpool_3D(Layer_10r4)
    Layer_10r4 =  tf.nn.dropout(Layer_10r4,keep_prob_skip)
    
    print(Layer_10r4)

In [ ]:
Layer_10 = tf.concat([tf.concat([tf.concat([Layer_10,Layer_10r2],-1),Layer_10r3],-1),Layer_10r4],-1)
Layer_10

# Output Layer

In [ ]:
Reuse_logits = False
with tf.variable_scope("Logits",reuse=Reuse_logits) as scope:
    
    logits = slim.conv3d(Layer_10, 2, 
                          kernel_size=[ 3, 3, 3],
                          stride=[1, 1, 1], 
                          activation_fn=None)
    logits = tf.nn.relu(logits)
    print(logits)

In [ ]:
# Re-define for 5D Tensor, here we are dealing with 5D: axis=3 become axis=4 last channel
def pixel_wise_softmax(output_map):
    return tf.nn.softmax(output_map)

In [ ]:
reuseflagOutputHM= False
with tf.variable_scope("output",reuse=reuseflagOutputHM) as scope:
    Segmentation_Maps = pixel_wise_softmax(logits)
    print(Segmentation_Maps)

In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    #print(shape)
    #print(len(shape))
    variable_parameters = 1
    for dim in shape:
        #print(dim)
        variable_parameters *= dim.value
    #print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

# Define the Loss based on Tversky Index and Learned Alpha and Beta

# compare with pseduo Code which is our previous estimation

In [ ]:
smooth=1
P_foreground_SOL = Segmentation_Maps[...,:1]
P_background_SOL  = Segmentation_Maps[...,1:]
g_foreground_SOL  = y[...,:1]
g_background_SOL  = y[...,1:]
print(P_foreground_SOL)
print(P_background_SOL)

# Compute the TP, FP, and FN for each Muscle

In [ ]:
#SOL
true_positive_SOL= P_foreground_SOL * g_foreground_SOL
#P_foreground_SOL = tf.nn.dropout(P_foreground_SOL,keep_prob_tp,name="P_foreground_drop_out_SOL")
false_pos_SOL = P_foreground_SOL * g_background_SOL
false_neg_SOL = P_background_SOL * g_foreground_SOL

print(true_positive_SOL)
print(false_pos_SOL)
print(false_neg_SOL)

In [ ]:
def weight_variable2(shape):
    weights_initializer=tf.random_uniform_initializer(minval=0.5, maxval=0.5, seed=None)
    Weight = tf.Variable(weights_initializer(shape=shape))    
    return Weight

In [ ]:
W_tanh = weight_variable2([1, 2])
W_tanh = tf.nn.softmax(W_tanh)
print(W_tanh)

# Penealise the FP and the FN by learned alpha and beta

In [ ]:
true_positive_SOL=tf.reduce_sum(tf.reduce_sum(true_positive_SOL, axis=(2,3,4)),axis=(1))
false_pos_SOL=tf.reduce_sum(W_tanh[0][0]*tf.reduce_sum(false_pos_SOL, axis=(2,3,4)),axis=(1))
false_neg_SOL=tf.reduce_sum(W_tanh[0][1]*tf.reduce_sum(false_neg_SOL, axis=(2,3,4)),axis=(1))
print(true_positive_SOL)
print(false_pos_SOL)
print(false_neg_SOL)

# Compute Tversky Index for each of the muscle

In [ ]:
TverskyIndex_SOL = tf.divide(true_positive_SOL+smooth, (true_positive_SOL + false_pos_SOL  + false_neg_SOL  + smooth))
print(TverskyIndex_SOL)    


In [ ]:

with tf.name_scope("loss"):
    parametricTverskyLoss = 1-tf.reduce_mean(TverskyIndex_SOL)
print(parametricTverskyLoss)


# Schaduale Learning rate

In [ ]:
# Schaduale Learning rate
global_step = tf.Variable(0, trainable=False)
boundaries = [6000]
values = [0.0001,0.00001]
learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)

In [ ]:
with tf.name_scope('Optimizer'):
    train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(parametricTverskyLoss,global_step=global_step)

# Computing Some Measures

In [ ]:
# Compute Dice Value
IoU_SOL = iou_coe(Segmentation_Maps,y)      
print(IoU_SOL)

In [ ]:
# Compute Dice Value
TverskyIndexValue_SOL = Tversky_Index(Segmentation_Maps,y)      
print(TverskyIndexValue_SOL)

In [ ]:
with tf.name_scope("Precision_Recall"):
    output_thres_SOL = tf.cast(tf.squeeze(Segmentation_Maps[...,:1],0)> 0.5, dtype=tf.float32)
    
    target_thres_SOL = tf.cast(tf.squeeze(y[...,:1],0)> 0.5, dtype=tf.float32)
    
    precVSOL, recallVSOL = confusion(target_thres_SOL,output_thres_SOL)

In [ ]:
#MeanP = tf.reduce_mean(precVSOL+precVGL+precVGM)
#MeanR = tf.reduce_mean(recallVSOL+recallVGL+recallVGM)

In [ ]:
saver = tf.train.Saver()
model_path = '/tf/2021Work/AnalysisiFilterSharingV2_3/Model/'

In [ ]:
import time
from IPython import display

In [ ]:
init_o = tf.global_variables_initializer()
Modelsummary={}
isRestor=False

# Run the Session

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pickle
from random import randint

In [ ]:
def compute_mean_iou(pred, label):

    unique_labels = np.unique(label)
    num_unique_labels = len(unique_labels);

    I = np.zeros(num_unique_labels)
    U = np.zeros(num_unique_labels)

    for index, val in enumerate(unique_labels):
        pred_i = pred == val
        label_i = label == val

        I[index] = float(np.sum(np.logical_and(label_i, pred_i)))
        U[index] = float(np.sum(np.logical_or(label_i, pred_i)))
    mean_iou = np.mean(I / U)
    return mean_iou

# run Session

In [ ]:
'''
incremental learning over the subjects with 50% 40 30 20 10 ...
'''

# use the 10 Patient as Training from the Test Set and the 1st 19 Patients from the training Set

In [ ]:
x_y_path = '/tf/volumes/test/'
Patient_folder_path = sorted(os.listdir(x_y_path))
Patient_folder_full_path = list(map(lambda v : str(join(x_y_path,v)) + '/', Patient_folder_path))

# Get the full path for the volume and the mask
DataTestPath = list(map(lambda s : s+'x1.mha' , Patient_folder_full_path))
MasksTestPath = list(map(lambda s : s+'masksX1.mha' , Patient_folder_full_path))

In [ ]:
#ValidationSet

In [ ]:
# Define the path 
x_y_path = '/tf/volumes/val/'
# define the full path for each patient
Patient_folder_path = sorted(os.listdir(x_y_path))
Patient_folder_full_path = list(map(lambda v : str(join(x_y_path,v)) + '/', Patient_folder_path))

# Get the full path for the volume and the mask
DataValPath = list(map(lambda s : s+'x1.mha' , Patient_folder_full_path))
MasksValPath = list(map(lambda s : s+'masksX1.mha' , Patient_folder_full_path))

In [ ]:
TrainSetDataPath = DataTestPath+DataTrainPath+DataValPath
TrainSetMaskPath = MasksTestPath+MasksTrainPath+MasksValPath

In [ ]:
len(TrainSetDataPath)

In [ ]:
def ReturnIndicesOfFullMask(sequence):
    '''
    The input sequence is of shape (1583, 512, 512)
    The output is a list of 2 indices, the begining and the end of thesequence with full masks
    Return the begining and the end of the sequence
    '''
    result = list(map(lambda img: img.sum() ,sequence ))
    resultIndex = list(map(lambda element: element>2000 ,result))
    Indices = [i for i, x in enumerate(resultIndex) if x]
    return Indices[0],Indices[-1]

In [ ]:
def SampleData(subjects): 
    StartEnd =list(map(lambda el1: ReturnIndicesOfFullMask(process_mask_GL(TrainSetMaskPath[el1])[:,:,:,0]),
                              subjects))
    indicesToSampleFrom = list(map(lambda el1:random.sample(range(el1[0],el1[1]-3), 1)[0], StartEnd))
    SampledIndicies = list(map(lambda el1:list(range(el1,el1+3)), indicesToSampleFrom))
    x_train = np.array(list(map(lambda a,b:Pull_data_from_path(TrainSetDataPath[a])[b],subjects,SampledIndicies)))
    y_train = np.array(list(map(lambda a,b:process_mask_GL(TrainSetMaskPath[a])[b],subjects,SampledIndicies)))
    return x_train,y_train

def Binaize(im):
    return cv2.threshold(im[...,0],0.5,1,cv2.THRESH_BINARY)[1].astype('uint8')

# Learning Process

## Total Number of Parameters

In [ ]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    #print(shape)
    #print(len(shape))
    variable_parameters = 1
    for dim in shape:
        #print(dim)
        variable_parameters *= dim.value
    #print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

# Run the Session

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(12,6))
fig.tight_layout(pad=1.5)

isRestor=False
file_path = model_path + 'model-GL'

with tf.Session() as sess:
    if isRestor:
        sess.run(init_o) 
        saver.restore(sess,file_path)
        print("Model restored.")
    else:
        sess.run(init_o) 
        print('Model Training from Scratch')
    
    start_time = time.process_time()
    
    itCounter=0
    
    train_Dice_Coef_list=[] 
    train_loss_list=[]      
    train_mIoU_list=[] 
    train_P_list=[] 
    train_R_list=[] 
    train_alpha_list = []
    train_beta_list = []
        
    batchsize=2
    for iterations in range(3001):
        SubjectToSampleFrom = random.sample(range(0,29), batchsize) # 3 mean batch of 8
        x_train,y_train =  SampleData(SubjectToSampleFrom)
        print('Input Shape: ', x_train.shape)

        # Feed the network to train with the Reference and the Target
        _, dice_losst,current_output,alpha,betta=sess.run([train_step,parametricTverskyLoss,
                                                                  Segmentation_Maps,W_tanh[0][0],W_tanh[0][1]],
                                                                 feed_dict={
                                                                     x: x_train,
                                                                     y: y_train,
                                                                     keep_prob:random.uniform(0.85, 1.),
                                                keep_prob_input:random.uniform(0.9, 1.),
                                                keep_prob_skip:random.uniform(0.85, 1.),
                                                keep_prob_alpha :1.0,
                                                keep_prob_beta :1.0,
                                                keep_prob_tp:0.95
                                               })
            
        im1 = current_output[0][-1][...,0].reshape(512,512)    
        
         # append the values over T segment and compute their mean for each patient   
        IOU = metric.jc(np.array(list(map(lambda a:list(map(lambda b:Binaize(b), a)), current_output))), y_train[...,0])
        DICE =  metric.dc(np.array(list(map(lambda a:list(map(lambda b:Binaize(b), a)), current_output))), y_train[...,0])
        Precision = metric.precision(np.array(list(map(lambda a:list(map(lambda b:Binaize(b), a)), current_output))), y_train[...,0])
        Recall = metric.recall(np.array(list(map(lambda a:list(map(lambda b:Binaize(b), a)), current_output))), y_train[...,0])

        print('compute the mean of the results')
            
        axs[0, 0].set_title('Tversky loss')
        axs[0, 1].set_title('mIoU')
        axs[0, 2].set_title('Dice Score')

        axs[1, 0].set_title('Alpha and Beta')
            
        axs[1, 1].set_title('P-R')
        axs[1, 2].set_title('Heat Map')
            
        axs[0, 0].plot([itCounter],dice_losst,'b*')
        axs[0, 1].plot([itCounter],IOU,'b*')
        axs[0, 2].plot([itCounter],DICE,'k*')                           
                                                
        axs[1, 0].plot([itCounter],alpha,'b*')
        axs[1, 0].plot([itCounter],betta,'g*')
            
        axs[1, 1].plot([itCounter],Precision,'b*')
        axs[1, 1].plot([itCounter],Recall,'k*')

        axs[1, 2].imshow(im1, aspect="auto",cmap='gist_earth')
            
        display.clear_output(wait=True)
        display.display(plt.gcf())   
        sys.stdout.flush()
            
        Modelsummary[itCounter]=[dice_losst, IOU,DICE,Precision,Recall,alpha,betta]  
            
        itCounter=itCounter+1

        print('next subject')
        
        if iterations%200==0:
            print('>> Model Saved and can be restored during another session!')
            saver.save(sess, file_path, global_step=iterations) 
            
            with open(model_path+'learningCurve.pickle', 'wb') as handle:
                    pickle.dump(Modelsummary, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print('>> Model Saved and can be restored during another session!')
    saver.save(sess, file_path, global_step=iterations) 
            
    with open(model_path+'learningCurve.pickle', 'wb') as handle:
        pickle.dump(Modelsummary, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('Done')